In [1]:
import pandas as pd
import pandahouse as ph

In [2]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

Задание 2. SQL 2.1 Очень усердные ученики.

2.1.1 Условие

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

In [3]:
q = '''
SELECT uniqExact(st_id) as uniq_id
FROM
    (SELECT st_id, 
            SUM(correct) AS sym_cor 
    FROM peas
    WHERE correct > 0
    GROUP BY st_id
    HAVING sym_cor>=20)

    '''

# отправляем запрос и записываем результат в пандасовский датафрейм
q_test = ph.read_clickhouse(query=q, connection=connection_default)
q_test

,uniq_id
0,136


2.2 Оптимизация воронки

2.2.1 Условие

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

In [4]:
q = '''

SELECT 
    SUM(fc.money)/uniqExact(s.st_id) as ARPU,
    SUM(fc.money)/uniqExact(ac.st_id) as ARPAU,
    ROUND((uniqExact(fc.st_id) / uniqExact(s.st_id)*100),2) as CR,
    ROUND((uniqExact(CASE WHEN fc.st_id IN (SELECT st_id
                                    FROM peas
                                    GROUP BY st_id
                                    HAVING COUNT(correct)>10) THEN fc.st_id END) / uniqExact(ac.st_id)*100),2) as CR_ACTIV,
    ROUND((uniqExact(CASE WHEN fc.st_id IN (SELECT st_id
                                    FROM peas
                                    WHERE subject = 'Math'
                                    GROUP BY st_id
                                    HAVING COUNT(correct) >= 2) THEN fc.st_id END) / uniqExact(ac_m.st_id)*100),2) as CR_MATH
    
FROM studs as s
LEFT JOIN (
    SELECT st_id
    FROM peas
    GROUP BY st_id
    HAVING COUNT(correct)>10) as ac
ON s.st_id = ac.st_id
        LEFT JOIN (
            SELECT st_id
            FROM peas
            WHERE subject = 'Math'
            GROUP BY st_id
            HAVING COUNT(correct) >= 2) as ac_m
        ON s.st_id=ac_m.st_id
                LEFT JOIN final_project_check as fc
                ON s.st_id = fc.st_id
GROUP BY s.test_grp
      

    '''

# отправляем запрос и записываем результат в пандасовский датафрейм.
q_test = ph.read_clickhouse(query=q, connection=connection_default)
q_test.head()

,ARPU,ARPAU,CR,CR_ACTIV,CR_MATH
0,4540.983607,9111.842105,5.25,9.87,11.76
1,11508.474576,27827.868852,11.19,21.31,13.33
